<a href="https://colab.research.google.com/github/Chirag314/W-B/blob/main/WnB_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Weight & Biases is a free Python library that allows you to track, compare, and visualize ML experiments. It is also integrated with many popular libraries such as TensorFlow, PyTorch, Kera, Scikit, Hugging Face, and XGBoost.

In [1]:
# Install W & B
!pip install wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=2d81015b5451af59a4b0a77a4b428bfc19dc5cbc796e7a540f9d371b2d10132f
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools


In [2]:
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [67]:
# Work on IRIS dataset
from sklearn.model_selection import cross_val_score
from sklearn import datasets
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression


In [68]:
#Initilize wandb project so that all outputs for this experiment will be saved under IRIS project
wandb.init(project='iris')

In [69]:
#Load iris dataset
import pandas as pd
iris_data=datasets.load_iris()
df = pd.DataFrame(data=iris_data.data,
                  columns=iris_data.feature_names)
target=pd.DataFrame(data=iris_data.target)



In [70]:
target.rename(columns={0:'target'},inplace=True)

In [71]:
df=df.join(target)

In [72]:
df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [73]:
X=iris_data.data
y=iris_data.target

In [74]:
wandb.log({"table":wandb.Table(data=df,columns=df.columns)})
wandb.finish()

In [75]:

#split into train and test set
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

model=LogisticRegression(solver='liblinear',multi_class='ovr')

kfold=StratifiedKFold(n_splits=10,random_state=42,shuffle=True)

#metrics
accuracy=cross_val_score(model,X_train,y_train,cv=kfold,scoring='accuracy').mean()
f1_macro=cross_val_score(model,X_train,y_train,cv=kfold,scoring='f1_macro').mean()
neg_log_loss=cross_val_score(model,X_train,y_train,cv=kfold,scoring='neg_log_loss').mean()

Log all metrics

In [76]:
wandb.init(project='iris')
wandb.log({'accuracy':accuracy,
           'f1_macro':f1_macro,
           'neg_log_loss':neg_log_loss})


This is just one experiment. We can add many experiments and check results.

In [77]:
def main(name_model,model):
  wandb.init(project='iris',
             group=name_model,
             reinit=True)
  # Load dataset
  df = datasets.load_iris()
  X = df.data
  y = df.target
  # Split into train and test set
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

  kfold = StratifiedKFold(n_splits=10, random_state=1, shuffle=True)
  accuracy = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy').mean()
  f1_macro = cross_val_score(model, X_train, y_train, cv=kfold, scoring='f1_macro').mean()
  neg_log_loss = cross_val_score(model, X_train, y_train, cv=kfold, scoring='neg_log_loss').mean()

  wandb.log({'accuracy': accuracy,
                'f1_macro': f1_macro,
                'neg_log_loss': neg_log_loss})




In [78]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB

In [79]:
if __name__=='__main__':
  models={"LogisticRegression":LogisticRegression(solver='liblinear',multi_class='ovr'),
          'LinearDiscriminantAnalysis':LinearDiscriminantAnalysis(),
          'KneighborsClassifier':KNeighborsClassifier(),
          'DecisionTreeClassifier':DecisionTreeClassifier(),
          'GaussianNB':GaussianNB()}

  for name,model in models.items():
    main(name,model)

accuracy,▁
f1_macro,▁
neg_log_loss,▁
accuracy,0.95833
f1_macro,0.95794
neg_log_loss,-0.33454


accuracy,▁
f1_macro,▁
neg_log_loss,▁
accuracy,0.94167
f1_macro,0.93782
neg_log_loss,-0.32999


accuracy,▁
f1_macro,▁
neg_log_loss,▁
accuracy,0.975
f1_macro,0.97337
neg_log_loss,-0.08502


accuracy,▁
f1_macro,▁
neg_log_loss,▁
accuracy,0.95833
f1_macro,0.95644
neg_log_loss,-0.37916


accuracy,▁
f1_macro,▁
neg_log_loss,▁
accuracy,0.94167
f1_macro,0.93545
neg_log_loss,-2.40291


Visualize Sklearn Models
wandb also allows us to create common plots to evaluate Sklearn models with built-in functions.

To visualize all classification plots,

In [80]:
# Load libraries
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import wandb

def main(name_model, model):

    wandb.init(project='iris',
                group=name_model, # Group experiments by model
    )

    # Load dataset
    df = datasets.load_iris()
    X = df.data
    y = df.target
    features = df.feature_names

    # Split into train and test set
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=1)

    # Train model, get predictions
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    y_probas = model.predict_proba(X_test)

    # Visualize all classification plots
    wandb.sklearn.plot_classifier(model,X_train,X_test,y_train,y_test,y_pred,y_probas,features,model_name=name_model)

if __name__=='__main__':
  main('LinearDiscriminantAnalysis',LinearDiscriminantAnalysis())

accuracy,▁
f1_macro,▁
neg_log_loss,▁
accuracy,0.95
f1_macro,0.94392
neg_log_loss,-0.14124


wandb: 
wandb: Plotting LinearDiscriminantAnalysis.
wandb: WARNING 2-dimensional feature importances array passed to plot_feature_importances. 2-dimensional and higher feature importances arrays are not currently supported. These importances will not be plotted.
wandb: Logged feature importances.
wandb: Logged confusion matrix.
wandb: Logged summary metrics.
wandb: Logged class proportions.
wandb: WARNING This function only supports binary classification at the moment and therefore expects labels to be binary. Skipping calibration curve.
wandb: Logged calibration curve.
wandb: Logged roc curve.
wandb: Logged precision-recall curve.


In [81]:
wandb.finish()